<a href="https://colab.research.google.com/github/EllisBuxton/PPIT-Project/blob/main/MusicGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Music Generator




*   I am going to using a genetic algorithm for the music generation
*   the user will rate each variation of the audio which will alter how much it changes



**The Algorithm:**

In [46]:
from random import choices, randint, randrange, random, sample
from typing import List, Optional, Callable, Tuple

Type Aliases (better maintainability) :

In [47]:
Genome = List[int] # Each genome will represent a specific soloution
Population = List[Genome] # A collection of genome's
PopulateFunc = Callable[[], Population] # Generates initial population of genome's
FitnessFunc = Callable[[Genome], int] # Evaluates how well it solves problem
SelectionFunc = Callable[[Population, FitnessFunc], Tuple[Genome, Genome]] # Selecting parent genome's based on fitness
CrossoverFunc = Callable[[Genome, Genome], Tuple[Genome, Genome]] #  Takes two genomes as input and returns a tuple of two genomes (used tombine parent genome's)
MutationFunc = Callable[[Genome], Genome] # Takes genome as input and return a modified genome
PrinterFunc = Callable[[Population, int, FitnessFunc], None] # Printing current state of population

Function to generate a radom genome of specified length:

In [ ]:
def generate_genome(length: int) -> Genome:
    return choices([0, 1], k=length)

Function to generate a population of genomes :

In [ ]:
def generate_population(size: int, genome_length: int) -> Population:
    return [generate_genome(genome_length) for _ in range(size)]

Function to perform crossover between two genomes:

In [ ]:
def single_point_crossover(a: Genome, b: Genome) -> Tuple[Genome, Genome]:
    if len(a) != len(b):
        raise ValueError("Genomes a and b must be of same length")

    length = len(a)
    if length < 2:
        return a, b

    # Select a random crossover point
    p = randint(1, length - 1)
    # Perform crossover and return offspring
    return a[0:p] + b[p:], b[0:p] + a[p:]

Function to perform mutation on a genome:

In [ ]:
def mutation(genome: Genome, num: int = 1, probability: float = 0.5) -> Genome:
    for _ in range(num):
        index = randrange(len(genome))
        # Flip the bit at the selected index with a certain probability
        genome[index] = genome[index] if random() > probability else abs(genome[index] - 1)
    return genome

Function to calculate the total fitness of a population :

In [ ]:
def population_fitness(population: Population, fitness_func: FitnessFunc) -> int:
    return sum([fitness_func(genome) for genome in population])

Function to select a pair of genomes from the population for reproduction:

In [ ]:
def selection_pair(population: Population, fitness_func: FitnessFunc) -> Population:
    return sample(
        population=generate_weighted_distribution(population, fitness_func),
        k=2
    )

Function to generate a weighted distribution of genomes based on their fitness:

In [ ]:
def generate_weighted_distribution(population: Population, fitness_func: FitnessFunc) -> Population:
    result = []

    for gene in population:
        result += [gene] * int(fitness_func(gene)+1)

    return result

Function to sort the population based on fitness:

In [ ]:
def sort_population(population: Population, fitness_func: FitnessFunc) -> Population:
    return sorted(population, key=fitness_func, reverse=True)

Function to convert a genome to a string for printing

In [ ]:
def genome_to_string(genome: Genome) -> str:
    return "".join(map(str, genome))

Function to print statistics of the current population

In [ ]:
def print_stats(population: Population, generation_id: int, fitness_func: FitnessFunc):
    print("GENERATION %02d" % generation_id)
    print("=============")
    print("Population: [%s]" % ", ".join([genome_to_string(gene) for gene in population]))
    print("Avg. Fitness: %f" % (population_fitness(population, fitness_func) / len(population)))
    sorted_population = sort_population(population, fitness_func)
    print(
        "Best: %s (%f)" % (genome_to_string(sorted_population[0]), fitness_func(sorted_population[0])))
    print("Worst: %s (%f)" % (genome_to_string(sorted_population[-1]),
                              fitness_func(sorted_population[-1])))
    print("")

    return sorted_population[0]